# Clustering the songs from dataframes

Now it's time to cluster the songs of the hot_songs and not_hot_songs databases according to the song's audio features. You will need to consider the following:

- Are you going to use all the audio features? If not, which ones do you think to make more sense?
- What is the optimal number of clusters (for methods that need to know this beforehand)?
- What is the best distance to use?
- What clustering method provides better results?
- Does the clustering method need a transformer?

Be aware that this process is extremely time-consuming!!! Therefore, when testing different options, save the models into your disk in order to be able to use the best model later.  You don't want to retrain the best model again when you know what are the optimal parameters for each.

Add to the hot_songs and not_hot_songs databases a new column for each clustering method with the cluster membership of each song for each method.

## Importing the libraries

In [ ]:
pip install sklearn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pickle

In [ ]:
all_songs = pd.read_csv('data/allsongsconcat_df.csv')

## Removing all the unnecessary audio features:
These are not actually audio features, just links and other kind of information that has nothing to do with audio qualities.

In [ ]:
all_songs_clean = all_songs.drop(['analysis_url', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'type'],axis =1)

## Store this cleaned Dataframe in a csv file:

In [ ]:
all_songs_clean.to_csv("data/all_songs_clean.csv", index=False)

## Numerical and Categorical split:
- X_num will be for Numerical columns
- X_cat will be for Categorical ones

In [ ]:
X_num = all_songs_clean.drop(['songs', 'artists'], axis =1)

In [ ]:
X_cat = all_songs[['songs', 'artists']]

## Checking the Data types per column

In [ ]:
X_num.info()

In [ ]:
X_cat.info()

## Scaling the features

In [ ]:
scaler = StandardScaler()
scaler.fit(X_num)#We will not use all the X, only the numerical features, select the appropriate num features
X_scaled = scaler.transform(X_num)
filename = "/Users/Hector_Martin/Documents/Labs/music_recommender_project/scalers/standardscaler.pickle" # Path with filename
with open(filename, "wb") as file:
        pickle.dump(scaler,file)
X_scaled_df = pd.DataFrame(X_scaled, columns = X_num.columns)
print('Data before the transformation')
print('------------------------------')
display(X_num.head())#data before the transformation
print()
print('Data after the transformation')
print('------------------------------')
display(X_scaled_df.head())#data after the transformation

## Training Models with different K values to assess which offers the best performance:

In [ ]:
def k_means_trainer(df):
    
    '''
    The formula trains several models and plots their performance using the Silhouette and the Elbow method.
    All models are stored in a pickle file.
    '''
    
    #We start with 2 because we need at least 2 groups to compare 
    #From 2 to 21 because we want to compare the performance of our models with up to 20 songs
    
    K = range(2, 21)
    inertia = [] #Store the inertia value of every model
    silhouette = [] #Store the silhouette score of every model

    for k in K:
        print("Training a K-Means model with {} neighbours! ".format(k))
        print()
        kmeans = KMeans(n_clusters=k,
                        n_init = 10, #Train 10 models, the function will store only 1 as a pickle file.
                        random_state=1234,
                        verbose =1) #Display progress messages
        kmeans.fit(df)
        filename = "/Users/Hector_Martin/Documents/Labs/music_recommender_project/models/kmeans_" + str(k) + ".pickle"
        with open(filename, "wb") as file:
            pickle.dump(kmeans,file)
        inertia.append(kmeans.inertia_)
        silhouette.append(silhouette_score(df, kmeans.predict(df)))


    import numpy as np
    import matplotlib.pyplot as plt
    %matplotlib inline

    #Elbow Method:
    fig, ax = plt.subplots(1,2,figsize=(16,8))
    ax[0].plot(K, inertia, 'bx-')
    ax[0].set_xlabel('k')
    ax[0].set_ylabel('inertia')
    ax[0].set_xticks(np.arange(min(K), max(K)+1, 1.0))
    ax[0].set_title('Elbow Method showing the optimal k')

    #Silhouette Method:
    ax[1].plot(K, silhouette, 'bx-')
    ax[1].set_xlabel('k')
    ax[1].set_ylabel('silhouette score')
    ax[1].set_xticks(np.arange(min(K), max(K)+1, 1.0))
    ax[1].set_title('Silhouette Method showing the optimal k')

In [ ]:
k_means_trainer(X_scaled_df)

### Loading the scaler and the best model:

In [ ]:
def load(filename = "filename.pickle"): #if I don't specify the name of the file it's going to be 'filename.pickle'
    try: 
        with open(filename, "rb") as file: 
            return pickle.load(file) 
    except FileNotFoundError: 
        print("File not found!") 

#### Loading the scaler from a pickle file

In [ ]:
scaler = load("scalers/standardscaler.pickle")
scaler

#### Loading the best_model from a pickle file:
Based on the Elbow method graphic we plotted, we can determine that the best model is the one with 8Ks.

In [ ]:
best_model = load("models/kmeans_8.pickle")